[![Image Name](https://cdn.kesci.com/upload/sehv2zq3qx.jpg?imageView2/0/w/960/h/960)](https://www.heywhale.com/home/competition/66598b3271a1fd975a17d6ad)  
[**vgbhfive**](http://blog.vgbhfive.com)，多年风控引擎研发及金融模型开发经验，现任某公司风控研发工程师，对数据分析、金融模型开发、风控引擎研发具有丰富经验。

在上一关中学习了如何训练决策树模型、可视化决策树、计算特征重要性、计算 `r2` 分数指标等内容，下面我们将学习今天的内容 `SVM` 支持向量机。

# SVM

`SVM` 支持向量机，指定义在特征空间上的间隔最大的线性分类器。单从名字来看该算法是最摸不着头脑的算法，不过在了解其定义之后可以明白其是对逻辑回归的演进，其重点在于**扩展特征维度空间**。  

在之前在逻辑回归问题中，会将日常问题抽象为二维特征空间内的问题，那如果将日常问题映射到三维特征空间，或者多维特征空间，会不会更好区分呢？  

而 `SVM` 的重点就是求解该**映射关系的参数**，映射关系在 `SVM` 中被称为**核函数**。  



目前可用的核函数有以下:  
 - 线性核（`Linear Kernel`）：适用于线性可分的数据。如果你的数据在特征空间中可以通过一条直线分割，线性核可能是一个好选择。它计算速度快，参数少，对于一般数据，分类效果已经很理想。  
 - 多项式核函数：适用于非线性问题，可以将特征映射到更高次幂的多项式空间。核函数的阶数由参数 `degree` 决定，决策边界为多项式的曲面。  
 - 径向基函数（`RBF`）核：又称高斯核，适用于线性不可分的数据。`RBF` 核可以将数据映射到一个高维空间，使得数据在新的空间中更容易分割。其具有更多的参数，并且分类结果非常依赖于参数的选择，通过训练数据的交叉验证来寻找合适的参数是常见的做法，但这个过程可能比较耗时。  
 - `Sigmoid` 核函数：类似于神经网络的激活函数，适用于需要逻辑回归模型的决策边界，即收拢最终的结果只在一定边界内。 

### 肥胖风险数据集之二分类问题

随着现在生活水平的提高，现在已经不必为了吃饱肚子而是为了更好吃的食物，因此导致的肥胖问题也更加严重。为了更好地理解肥胖的成因，预测个体的肥胖风险，并为预防和治疗提供支持，现计划开发一个基于机器学习的肥胖风险评估模型。该数据集收集了大量人群的健康数据，共有20758 例记录，其中包括年龄、性别、体重、身高、家族肥胖史、身体活动量、饮食习惯、运动方式等特征信息。

肥胖风险数据集数据含义如下：  

|特征列名称	| 特征含义 |  
| ----- | ----- |  
| Gender	| 性别  |  
| Age	| 年龄  |  
| Height	| 身高  |  
| Weight	| 体重  |  
| family_history_with_overweight	| 家族肥胖史  |  
| FAVC	| 是否频繁食用高热量食物  |  
| FCVC	| 食用蔬菜的频次  |  
| NCP	| 食用主餐的次数  |  
| CAEC	| 两餐之间的食品消费：always（总是）；frequently（经常）；sometimes（有时候）  |  
| SMOKE	| 是否吸烟  |  
| CH2O	| 每日耗水量  |  
| SCC	| 高热量饮料消耗量  |  
| FAF	| 运动频率  |  
| TUE	| 使用电子设备的时间  |  
| CALC	| 酒精消耗量：0（无）； frequently（经常）；sometimes（有时候）  |  
| MTRANS	| 日常交通方式：Automobile（汽车）；Bike（自行车）；Motorbike（摩托车）；Public Transportation（公共交通）；Walking（步行）  |  
| 0be1dad	| 肥胖水平 |  


#### 引入依赖

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

#### 加载数据

In [3]:
# 1. 加载数据

obesity = pd.read_csv('/home/mw/input/obesity5367/obesity_level.csv', index_col='id')
obesity.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,0be1dad
id,,,,,,,,,,,,,,,,,
0,Male,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,Sometimes,0,2.763573,0,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,Frequently,0,2.000000,0,1.000000,1.000000,0,Automobile,0rmal_Weight
2,Female,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,Sometimes,0,1.910378,0,0.866045,1.673584,0,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,Sometimes,0,1.674061,0,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,Sometimes,0,1.979848,0,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


#### 数据基础分析

In [4]:
# 2. 数据基础分析

obesity.info()
# 数据项各列不存在空值

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20758 entries, 0 to 20757
Data columns (total 17 columns):
Gender                            20758 non-null object
Age                               20758 non-null float64
Height                            20758 non-null float64
Weight                            20758 non-null float64
family_history_with_overweight    20758 non-null int64
FAVC                              20758 non-null int64
FCVC                              20758 non-null float64
NCP                               20758 non-null float64
CAEC                              20758 non-null object
SMOKE                             20758 non-null int64
CH2O                              20758 non-null float64
SCC                               20758 non-null int64
FAF                               20758 non-null float64
TUE                               20758 non-null float64
CALC                              20758 non-null object
MTRANS                            20758 non-nul

In [6]:
# 3. 结果列的分布情况

obesity['0be1dad'].value_counts()

Obesity_Type_III       4046
Obesity_Type_II        3248
0rmal_Weight           3082
Obesity_Type_I         2910
Insufficient_Weight    2523
Overweight_Level_II    2522
Overweight_Level_I     2427
Name: 0be1dad, dtype: int64

#### 特征预处理

In [7]:
# 4. 对分类型特征值进行编码

lb = LabelEncoder()
columns = ['Gender', 'CAEC', 'CALC', 'MTRANS', '0be1dad']
for col in columns:
    obesity[col] = lb.fit_transform(obesity[col])

In [8]:
# 5. 查看各个特征与结果列的相关性

obesity.corr()['0be1dad']

Gender                            0.033655
Age                               0.269122
Height                            0.073753
Weight                            0.410058
family_history_with_overweight    0.298750
FAVC                              0.016886
FCVC                              0.054972
NCP                              -0.089360
CAEC                              0.175111
SMOKE                            -0.008864
CH2O                              0.182406
SCC                              -0.051350
FAF                              -0.097437
TUE                              -0.056894
CALC                              0.124926
MTRANS                           -0.081371
0be1dad                           1.000000
Name: 0be1dad, dtype: float64

In [9]:
# 6. 拆分特征列和结果列

y = obesity['0be1dad']
x = obesity.drop('0be1dad', axis=1)
x.head(), y.head()

(    Gender        Age    Height      Weight  family_history_with_overweight  \
 id                                                                            
 0        1  24.443011  1.699998   81.669950                               1   
 1        0  18.000000  1.560000   57.000000                               1   
 2        0  18.000000  1.711460   50.165754                               1   
 3        0  20.952737  1.710730  131.274851                               1   
 4        1  31.641081  1.914186   93.798055                               1   
 
     FAVC      FCVC       NCP  CAEC  SMOKE      CH2O  SCC       FAF       TUE  \
 id                                                                             
 0      1  2.000000  2.983297     3      0  2.763573    0  0.000000  0.976473   
 1      1  2.000000  3.000000     2      0  2.000000    0  1.000000  1.000000   
 2      1  1.880534  1.411685     3      0  1.910378    0  0.866045  1.673584   
 3      1  3.000000  3.000000    

#### 训练模型

In [10]:
# 7. 划分训练集和测试集 7:3

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_train.head(), x_test.head(), y_train.head(), y_test.head()

(       Gender        Age    Height      Weight  \
 id                                               
 1846        1  32.000000  1.750000  120.000000   
 14225       0  19.783234  1.822573  133.952675   
 9438        1  21.793724  1.776989   89.993812   
 12459       0  21.000000  1.670000   66.000000   
 12189       0  25.470652  1.628205  107.378702   
 
        family_history_with_overweight  FAVC      FCVC       NCP  CAEC  SMOKE  \
 id                                                                             
 1846                                1     0  2.000000  3.000000     3      0   
 14225                               1     1  3.000000  3.000000     3      0   
 9438                                1     1  2.921225  2.983201     3      0   
 12459                               1     1  2.000000  3.000000     3      0   
 12189                               1     1  3.000000  3.000000     3      0   
 
            CH2O  SCC       FAF       TUE  CALC  MTRANS  
 id           

In [11]:
# 8. 构建SVM 模型

svc = svm.SVC(kernel='linear', C=1, gamma='auto')
svc.fit(x_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

#### 如何确定核函数？

In [19]:
# 8.1 在遇到完全陌生的数据集，如何选择 SVM 模型最优核函数？当然是最简单的方法，对不同的核函数进行尝试，根据最后计算得分最高的就是最优核函数

# 定义不同的核函数和参数进行尝试
kernels = ['linear', 'rbf', 'poly', 'sigmoid']

# 存储每个参数组合的交叉验证分数
scores = []

for kernel in kernels:
    svc = svm.SVC(kernel=kernel, C=1, gamma='auto')
    scores.append(cross_val_score(svc, x_train, y_train, cv=5).mean())

scores

`sklearn.svm.SVC()` 支持向量机参数详解，用法如下：  
```python  
class sklearn.svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)  
```

挑选重要性较大的几个参数说明：  
 - `C`：浮点数，可不填，默认1.0，必须大于等于 `0`，为松弛系数的惩罚项系数。  
   如果 `C` 值设定比较大，那 `SVC` 可能会选择边际较小的，能够更好地分类所有训练点的决策边界。如果 `C` 的设定值较小,那 `SVC` 会尽量最大化边界,决策功能会更简单, 但代价是训练的准确度。  总而言之, `C` 在 `SVM` 中的影响就像正则化参数对逻辑回归的影响。  
 - `kernel`：字符，可不填，默认 `rbf`。指定要在算法中使用的核函数类型,可以输入 `linear`, `poly`, `rbf`, `sigmoid`。   
 - `degree`：整数，可不填，默认 `3`。多项式核函数的次数(`poly`) ,如果核函数没有选择 `poly`,这个参数会被忽略。  
 - `gamma`：浮点数，可不填，默认 `auto`。核函数的系数,仅在参数 `Kernel` 的选项为 `rbf`、`poly` 和 `sigmoid` 的时候有效。当输入 `auto`, 自动使用 `1/(n_features)`作为 `gamma` 的取值。  
 - `coefo`：浮点数，可不填，默认 `0.0`。核函数中的独立项,它只在参数 `kernel` 为 `poly`和 `sigmoid` 的时候有效。

#### 预测测试集并计算指标

In [17]:
# 9. 预测测试集

y_pred = svc.predict(x_test)
y_pred

In [18]:
# 10. 计算测试集的平均准确率和预测测试集准确率

acc = accuracy_score(y_test, y_pred)

svc.score(x_test, y_test), acc

#### 总结  
`SVM` 支持向量机采用扩展维度空间的方式进行分类，从而避免了之前逻辑回归的二维空间内的问题（线性不可分）。`SVM` 在扩展维度空间后，即当前数据线性可分，通过计算间隔最大化的分离超平面将数据分开，其对未知数据的预测性是最强的。

### 课后思考题

1. 当训练数据线性不可分时，为何引入核函数就可以可分？（对偶问题）

### 闯关题

#### STEP1：请根据要求完成题目

Q1. svm中的核函数下面解释正确的是？（多选题）  
   A. `linear` 线性分类核函数，适用于线性可分的数据。  
   B. `poly` 适用于非线性的问题。  
   C. `rbf` 适用于线性不可分的数据。  
   D. `sigmoid` 适用于需要逻辑回归模型的决策边界。

Q2. SVM出现欠拟合时，下面哪些可以解决？（欠拟合是指模型不能在训练集上获得足够低的误差，即模型训练较少，没有充分从训练集中学习到规律。）  
    A. 增大惩罚参数 C 的值  
    B. 减小惩罚参数 C 的值  
    C. 减小核系数（gamma参数）

Q3. 使用iris 数据集基于SVM 训练模型计算其准确率是多少？  
   A. 1  
   B. 0.99  
   C. 0.98  
   D. 0.97

In [5]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# 加载数据
iris = load_iris()
x, y = iris.data, iris.target
# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
# x_train.head(), x_test.head(), y_train.head(), y_test.head()
# 训练模型
# 定义不同的核函数和参数进行尝试
kernels = ['linear', 'rbf', 'poly', 'sigmoid']

# 存储每个参数组合的交叉验证分数
scores = []

for kernel in kernels:
    svc = svm.SVC(kernel=kernel, C=1, gamma='auto')
    scores.append(cross_val_score(svc, x_train, y_train, cv=5).mean())

print(scores)


[0.9609090909090909, 0.9518181818181818, 0.9531225296442688, 0.3522924901185771]


In [9]:
# 预测测试集
svc = svm.SVC(kernel='poly', C=1, gamma='auto')
svc.fit(x_train, y_train)
y_pred = svc.predict(x_test)
# 计算准确率指标
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [13]:
#填入你的答案并运行,注意大小写
a1 = 'ABCD'  # 如 a1= 'A'
a2 = 'A'  # 如 a2= 'A'
a3 = 'A'  # 如 a3= 'A'

#### STEP2：将结果保存为 csv 文件  
将结果保存为 csv 文件  
csv 需要有两列，列名：id、answer。其中，id 列为题号，如 q1、q2；answer 列为 STEP1 中各题你计算出来的结果。💡 这一步的代码你不用做任何修改，直接运行即可。

In [14]:
import pandas as pd

# 生成 csv 作业答案文件
def save_csv(a1, a2, a3):
    df = pd.DataFrame({"id": ["q1", "q2", "q3"], "answer": [a1, a2, a3]})
    df.to_csv("answer_4.csv", index=None)
    print(df)

save_csv(a1, a2, a3)

   id answer
0  q1   ABCD
1  q2      A
2  q3      A


#### STEP3: 提交 csv 文件，获取分数结果  
提交 csv 文件，获取分数结果  

你的 csv 答案文件已经准备完毕了，最后让我们提交答案文件，看看是否正确。  

提交方法：  

1、拷贝提交 token  

去对应关卡的 提交页面，找到对应关卡，看到了你的 token 嘛？  

拷贝它。  

记得：每个关卡的 token 不一样。  

2、下方 cell 里，拿你拷贝的 token 替换掉 XXXXXXX， 然后 Ctrl+Enter 运行 。

In [15]:
#运行这个Cell 下载提交工具

!wget -nv -O heywhale_submit https://cdn.kesci.com/submit_tool/v4/heywhale_submit&&chmod +x heywhale_submit

# 运行提交工具
# 把下方XXXXXXX替换为你的 Token
# 改完看起来像是：!./heywhale_submit -token 586eeef71cb92941 -file answer_4.csv

!./heywhale_submit -token 24fe3b30539c8fc1 -file answer_4.csv  # 替换XXXXXXX；注意不可增减任何空格或其他字符

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
2025-07-18 04:41:32 URL:https://cdn.kesci.com/submit_tool/v4/heywhale_submit [22020102/22020102] -> "heywhale_submit" [1]
Heywhale Submit Tool: v5.0.1 
> 已验证Token
> 开始上传文件
28 / 28 [||||||||||||||||||||||||||||||||||||||||||||||||||||||||] ? p/s 100.00%
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
